# Dependencies

In [ ]:
!pip install face_recognition

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.1/100.1 MB 10.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for face-recognition-models: filename=face_recognition_models-0.3.0-py2.py3-none-any.whl size=100566170 sha256=93212ebf9f6eb09b3fa4757faca837e7e3f7dcc1822777987ccbeabacbe29987
  Stored in directory: /root/.cache/pip/wheels/7a/eb/cf/e9eced74122b679557f597bb7c8e4c739cfcac526db1fd523d
Successfully built face-recognition-models


In [ ]:
!pip install face_recognition         # Install face_recognition package using pip
import face_recognition
import cv2                            # Import OpenCV
import numpy as np
from matplotlib import pyplot as plt  # pyplot

# ipywidgets packages
import ipywidgets as widgets
from IPython.display import display, clear_output

# Live Streaming dependencies
from IPython.display import Javascript, Image
from google.colab.output import eval_js
from base64 import b64decode, b64encode
import PIL
import io
import html
import time

# **Utils**

### **In-memory data**

In [ ]:
registered_faces = {}

# **face recognition**

In [ ]:
def check_if_face_registered(face_encoding):
  for _, registered_face in registered_faces.items():
    if True in face_recognition.compare_faces([face_encoding], registered_face):
      return True
  return False

# **Real time streaming**

In [ ]:
# function to convert the JavaScript object into an OpenCV image
def js_to_image(js_reply):
  """
  Params:
          js_reply: JavaScript object containing image from webcam
  Returns:
          img: OpenCV BGR image
  """
  # decode base64 image
  image_bytes = b64decode(js_reply.split(',')[1])
  # convert bytes to numpy array
  jpg_as_np = np.frombuffer(image_bytes, dtype=np.uint8)
  # decode numpy array into OpenCV BGR image
  img = cv2.imdecode(jpg_as_np, flags=1)

  return img

# function to convert OpenCV Rectangle bounding box image into base64 byte string to be overlayed on video stream
def bbox_to_bytes(bbox_array):
  """
  Params:
          bbox_array: Numpy array (pixels) containing rectangle to overlay on video stream.
  Returns:
        bytes: Base64 image byte string
  """
  # convert array into PIL image
  bbox_PIL = PIL.Image.fromarray(bbox_array, 'RGBA')
  iobuf = io.BytesIO()
  # format bbox into png for return
  bbox_PIL.save(iobuf, format='png')
  # format return string
  bbox_bytes = 'data:image/png;base64,{}'.format((str(b64encode(iobuf.getvalue()), 'utf-8')))

  return bbox_bytes

def video_frame(label, bbox):
  data = eval_js('stream_frame("{}", "{}")'.format(label, bbox))
  return data

# JavaScript to properly create our live video stream using our webcam as input
def video_stream():
  js = Javascript('''
    var video;
    var div = null;
    var stream;
    var captureCanvas;
    var imgElement;
    var labelElement;

    var pendingResolve = null;
    var shutdown = false;

    function removeDom() {
       stream.getVideoTracks()[0].stop();
       video.remove();
       div.remove();
       video = null;
       div = null;
       stream = null;
       imgElement = null;
       captureCanvas = null;
       labelElement = null;
    }

    function onAnimationFrame() {
      if (!shutdown) {
        window.requestAnimationFrame(onAnimationFrame);
      }
      if (pendingResolve) {
        var result = "";
        if (!shutdown) {
          captureCanvas.getContext('2d').drawImage(video, 0, 0, 800, 600);
          result = captureCanvas.toDataURL('image/jpeg', 0.8)
        }
        var lp = pendingResolve;
        pendingResolve = null;
        lp(result);
      }
    }

    async function createDom() {
      if (div !== null) {
        return stream;
      }

      div = document.createElement('div');
      div.style.border = '2px solid black';
      div.style.padding = '3px';
      div.style.width = '100%';
      div.style.maxWidth = '600px';
      document.body.appendChild(div);

      const modelOut = document.createElement('div');
      modelOut.innerHTML = "<span>Status:</span>";
      labelElement = document.createElement('span');
      labelElement.innerText = 'No data';
      labelElement.style.fontWeight = 'bold';
      modelOut.appendChild(labelElement);
      div.appendChild(modelOut);

      video = document.createElement('video');
      video.style.display = 'block';
      video.width = div.clientWidth - 6;
      video.setAttribute('playsinline', '');
      video.onclick = () => { shutdown = true; };
      stream = await navigator.mediaDevices.getUserMedia(
          {video: { facingMode: "environment"}});
      div.appendChild(video);

      imgElement = document.createElement('img');
      imgElement.style.position = 'absolute';
      imgElement.style.zIndex = 1;
      imgElement.onclick = () => { shutdown = true; };
      div.appendChild(imgElement);

      const instruction = document.createElement('div');
      instruction.innerHTML =
          '<span style="color: red; font-weight: bold;">' +
          'When finished, click here or on the video to stop this demo</span>';
      div.appendChild(instruction);
      instruction.onclick = () => { shutdown = true; };

      video.srcObject = stream;
      await video.play();

      captureCanvas = document.createElement('canvas');
      captureCanvas.width = 800; //video.videoWidth;
      captureCanvas.height = 600; //video.videoHeight;
      window.requestAnimationFrame(onAnimationFrame);

      return stream;
    }
    async function stream_frame(label, imgData) {
      if (shutdown) {
        removeDom();
        shutdown = false;
        return '';
      }

      var preCreate = Date.now();
      stream = await createDom();

      var preShow = Date.now();
      if (label != "") {
        labelElement.innerHTML = label;
      }

      if (imgData != "") {
        var videoRect = video.getClientRects()[0];
        imgElement.style.top = videoRect.top + "px";
        imgElement.style.left = videoRect.left + "px";
        imgElement.style.width = videoRect.width + "px";
        imgElement.style.height = videoRect.height + "px";
        imgElement.src = imgData;
      }

      var preCapture = Date.now();
      var result = await new Promise(function(resolve, reject) {
        pendingResolve = resolve;
      });
      shutdown = false;

      return {'create': preShow - preCreate,
              'show': preCapture - preShow,
              'capture': Date.now() - preCapture,
              'img': result};
    }
    ''')

  display(js)

In [ ]:
# JavaScript code to stop the webcam stream
js_stop_stream = """
    const stream = video.srcObject;
    const tracks = stream.getTracks();

    tracks.forEach(function(track) {
        track.stop();
    });

    video.srcObject = null;
    div.style.display = 'none';  // Hide the container
"""

# Function to stop the video streaming
def stop_video_stream():
    display(Javascript(js_stop_stream))

# Register Faces

In [ ]:
# Create a text input widget for the name
name_input = widgets.Text(
    placeholder='Enter your name',
    description='Name:',
    disabled=False
)

# Create a file upload widget
upload_button = widgets.FileUpload(
    accept='image/*',
    multiple=False,
    description='Upload Image'
)

or_text = widgets.HTML(
    value=" or "
)

# Create a button to trigger face registration
register_button = widgets.Button(description='Register Face')

# Create a button to trigger image capture
capture_button = widgets.Button(description='Capture Image')

# Create an output widget to display the uploaded or captured face image
output = widgets.Output()

# Initialize variables to store the uploaded image and name
uploaded_image_data = None
registered_name = None

# Define a function to handle image upload
def handle_upload(change):
    global uploaded_image_data
    uploaded_filename = list(upload_button.value.keys())[0]
    uploaded_image_data = upload_button.value[uploaded_filename]['content']

    with output:
        clear_output()
        display(widgets.Image(value=uploaded_image_data))

    # Assuming uploaded_image_data is binary image data in bytes
    image_array = np.frombuffer(uploaded_image_data, np.uint8)
    image = cv2.imdecode(image_array, cv2.IMREAD_COLOR)

    # Convert the image to RGB format
    uploaded_image_data = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

def handle_capture(button):
  global uploaded_image_data
  # start streaming video from webcam
  video_stream()
  js_reply = video_frame("Capturing...", "")

  stop_video_stream()  # Stop the video stream after capturing the frame

  if not js_reply:
    with output:
      clear_output()
      print("Error capturing image!")

  # convert JS response to OpenCV Image
  uploaded_image_data = js_to_image(js_reply["img"])

  with output:
    clear_output()
    # Encode the captured frame as PNG
    _, buffer = cv2.imencode('.png', uploaded_image_data)

    # Convert the encoded image to bytes
    image_bytes = buffer.tobytes()
    display(widgets.Image(value=image_bytes))

# Define a function to handle face registration
def handle_register(button):
    global registered_name

    if uploaded_image_data is not None:
        registered_name = name_input.value

        # Detect faces in the frame
        face_locations = face_recognition.face_locations(uploaded_image_data)

        # If there is no face, raise a popup window
        if len(face_locations) == 0:
            with output:
              clear_output()
              print("No face detected in the image!")
              return

        # If there is more than one face, raise a popup window
        if len(face_locations) > 1:
            with output:
              clear_output()
              print("More than one face detected in the image!")
              return

        # Get the face encodings
        face_encodings = face_recognition.face_encodings(uploaded_image_data)

        # Check if the face is registered
        if check_if_face_registered(face_encodings[0]):
            with output:
              clear_output()
              print("Face already registered!")
              return

        with output:
            clear_output()
            print(f"Face registered for {registered_name}")
            registered_faces[registered_name] = face_encodings[0]

upload_button.observe(handle_upload, names='value')
capture_button.on_click(handle_capture)
register_button.on_click(handle_register)

# Create a container for the widgets
container = widgets.VBox([name_input, widgets.HBox([upload_button, or_text, capture_button]), register_button, output])

# Display the container
display(container)

# Recognize Faces

In [ ]:
# start streaming video from webcam
video_stream()
# label for video
label_html = 'Capturing...'
bbox = ''

while True:
  js_reply = video_frame(label_html, bbox)
  if not js_reply:
    break

  # convert JS response to OpenCV Image
  frame = js_to_image(js_reply["img"])

  # Get the face locations from the frame
  face_locations = face_recognition.face_locations(frame)

  # For each face in the frame
  for (top, right, bottom, left) in face_locations:
      # Draw a rectangle around the face
      cv2.rectangle(
          frame,
          (left, top),
          (right, bottom),
          (0, 0, 255),
          2,
      )
      # Get the face encoding of the face
      face_encoding = face_recognition.face_encodings(frame, [(top, right, bottom, left)])[0]

      # Compare the face encoding to the faces in the database
      matches = face_recognition.compare_faces(list(registered_faces.values()), face_encoding)

      # If there is a match
      if True in matches:
          # Get the index of the match
          match_index = matches.index(True)

          # Get the name of the match
          match_name = list(registered_faces.keys())[match_index]

          # Draw the name of the match on the frame
          cv2.putText(
              frame,
              match_name,
              (left, top - 10),
              cv2.FONT_HERSHEY_SIMPLEX,
              1,
              (0, 0, 255),
              2,
              cv2.LINE_AA,
          )
      else:
          # Draw unknown on the frame
          cv2.putText(
              frame,
              "Unknown",
              (left, top - 10),
              cv2.FONT_HERSHEY_SIMPLEX,
              1,
              (0, 0, 255),
              2,
              cv2.LINE_AA,
          )

  # Show image
  plt.imshow(frame)
  plt.show()

# **PROYECTO**

In [ ]:
import cv2
import numpy as np
import face_recognition
from IPython.display import display, clear_output
import ipywidgets as widgets
import pickle
from IPython.display import HTML, display

# DEFINICION DE CLASES
class Persona:
    def __init__(self, identificacion, nombre, fecha_nacimiento, nacionalidad, telefonos, emails, direccion, fotosUrl=[]):
        self.identificacion = identificacion
        self.nombre = nombre
        self.fecha_nacimiento = fecha_nacimiento
        self.nacionalidad = nacionalidad
        self.telefonos = telefonos
        self.emails = emails
        self.direccion = direccion
        self.fotosUrl = fotosUrl

class Profesor(Persona):
    def __init__(self, identificacion, nombre, fecha_nacimiento, nacionalidad, telefonos, emails, direccion, especialidad, anos_ensenanza, genero_profesor, fotosUrl=[]):
        super().__init__(identificacion, nombre, fecha_nacimiento, nacionalidad, telefonos, emails, direccion, fotosUrl)
        self.especialidad = especialidad
        self.anos_ensenanza = anos_ensenanza
        self.genero_profesor = genero_profesor

class Estudiante(Persona):
    def __init__(self, identificacion, nombre, fecha_nacimiento, nacionalidad, telefonos, emails, direccion, carrera, pasatiempos, signo_zodiacal, fotosUrl=[]):
        super().__init__(identificacion, nombre, fecha_nacimiento, nacionalidad, telefonos, emails, direccion, fotosUrl)
        self.carrera = carrera
        self.pasatiempos = pasatiempos
        self.signo_zodiacal = signo_zodiacal

class Miscelaneo(Persona):
    def __init__(self, identificacion, nombre, fecha_nacimiento, nacionalidad, telefonos, emails, direccion, turno, estado_civil, horas_trabajadas, fotosUrl=[]):
        super().__init__(identificacion, nombre, fecha_nacimiento, nacionalidad, telefonos, emails, direccion, fotosUrl)
        self.turno = turno
        self.estado_civil = estado_civil
        self.horas_trabajadas = horas_trabajadas

# Define and implement classes for Aula, Curso, and Asistencia
class Aula:
    def __init__(self, numero_aula, miscelaneo_asignado, otras_propiedades=[]):
        self.numero_aula = numero_aula
        self.miscelaneo_asignado = miscelaneo_asignado
        self.otras_propiedades = otras_propiedades

    # Add any additional methods as needed

class Curso:
    def __init__(self, codigo_curso, nombre_curso, periodo, carrera, estudiantes_matriculados, profesores, horario, aulas, otras_propiedades=[]):
        self.codigo_curso = codigo_curso
        self.nombre_curso = nombre_curso
        self.periodo = periodo
        self.carrera = carrera
        self.estudiantes_matriculados = estudiantes_matriculados
        self.profesores = profesores
        self.horario = horario
        self.aulas = aulas
        self.otras_propiedades = otras_propiedades

    # Add any additional methods as needed

class Asistencia:
    def __init__(self, nombre_persona, path_imagen, fecha_hora_reporte, otras_propiedades=[]):
        self.nombre_persona = nombre_persona
        self.path_imagen = path_imagen
        self.fecha_hora_reporte = fecha_hora_reporte
        self.otras_propiedades = otras_propiedades


# widget # widget # widget # widget # widget # idget # widget # widget # widget # widget # widget # widget #

def add_horizontal_line():
    display(HTML("<hr/>"))



add_horizontal_line()
add_horizontal_line()

# Create a text input widget for the name
# name_input = widgets.Text(
#     placeholder='Enter your name',
#     description='Name:',
#     disabled=False
# )


# Create a file upload widget
upload_button = widgets.FileUpload(
    accept='image/*',
    multiple=False,
    description='Upload Image'
)

or_text = widgets.HTML(
    value=" or "
)

# Create a button to trigger face registration
register_button = widgets.Button(description='Register Face')

# Create a button to trigger image capture
capture_button = widgets.Button(description='Capture Image')

# Create an output widget to display the uploaded or captured face image
output = widgets.Output()

# Initialize variables to store the uploaded image and name
uploaded_image_data = None
registered_name = None

# Initialize a dictionary to store registered faces
registered_faces = {}

# Define a function to handle image upload
def handle_upload(change):
    global uploaded_image_data
    uploaded_filename = list(upload_button.value.keys())[0]
    uploaded_image_data = upload_button.value[uploaded_filename]['content']

    with output:
        clear_output()
        display(widgets.Image(value=uploaded_image_data))

    # Assuming uploaded_image_data is binary image data in bytes
    image_array = np.frombuffer(uploaded_image_data, np.uint8)
    image = cv2.imdecode(image_array, cv2.IMREAD_COLOR)

    # Convert the image to RGB format
    uploaded_image_data = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

upload_button.observe(handle_upload, names='value')

# Define a function to handle image capture
def handle_capture(button):
    global uploaded_image_data
    # start streaming video from webcam
    video_stream()
    js_reply = video_frame("Capturing...", "")

    stop_video_stream()  # Stop the video stream after capturing the frame

    if not js_reply:
        with output:
            clear_output()
            print("Error capturing image!")

    # convert JS response to OpenCV Image
    uploaded_image_data = js_to_image(js_reply["img"])

    with output:
        clear_output()
        # Encode the captured frame as PNG
        _, buffer = cv2.imencode('.png', uploaded_image_data)

        # Convert the encoded image to bytes
        image_bytes = buffer.tobytes()
        display(widgets.Image(value=image_bytes))

capture_button.on_click(handle_capture)

# Define a function to handle face registration
def handle_register(button):
    global registered_name

    if uploaded_image_data is not None:
        registered_name = name_input.value

        # Detect faces in the frame
        face_locations = face_recognition.face_locations(uploaded_image_data)

        # If there is no face, raise a popup window
        if len(face_locations) == 0:
            with output:
                clear_output()
                print("No face detected in the image!")
                return

        # If there is more than one face, raise a popup window
        if len(face_locations) > 1:
            with output:
                clear_output()
                print("More than one face detected in the image!")
                return

        # Get the face encodings
        face_encodings = face_recognition.face_encodings(uploaded_image_data)

        # Check if the face is registered
        if check_if_face_registered(face_encodings[0]):
            with output:
                clear_output()
                print("Face already registered!")
                return

        with output:
            clear_output()
            print(f"Face registered for {registered_name}")
            registered_faces[registered_name] = face_encodings[0]

register_button.on_click(handle_register)

# Create a container for the widgets
container = widgets.VBox([name_input, widgets.HBox([upload_button, or_text, capture_button]), register_button, output])

# Display the container
display(container)


add_horizontal_line()
add_horizontal_line()


#***************************************************************************************************
#***************************************************************************************************
#***************************************************************************************************

# CREAR LOS WIDGETS PARA INSERTAR EL PROFESOR
professor_identification_input = widgets.Text(
    placeholder='Enter ID',
    description='ID:',
    disabled=False
)

professor_name_input = widgets.Text(
    placeholder='Enter name',
    description='Name:',
    disabled=False
)

professor_birthdate_input = widgets.Text(
    placeholder='Enter birthdate',
    description='Birthdate:',
    disabled=False
)

professor_nationality_input = widgets.Text(
    placeholder='Enter nationality',
    description='Nationality:',
    disabled=False
)

professor_telephones_input = widgets.Text(
    placeholder='Enter telephones',
    description='Telephones:',
    disabled=False
)

professor_emails_input = widgets.Text(
    placeholder='Enter emails',
    description='Emails:',
    disabled=False
)

professor_address_input = widgets.Text(
    placeholder='Enter address',
    description='Address:',
    disabled=False
)

professor_specialty_input = widgets.Text(
    placeholder='Enter specialty',
    description='Specialty:',
    disabled=False
)

professor_teaching_years_input = widgets.IntText(
    value=0,
    description='Teaching Years:'
)

professor_gender_input = widgets.Dropdown(
    options=['Male', 'Female', 'Other'],
    value='Male',
    description='Gender:'
)

# Create a button to register the professor
register_professor_button = widgets.Button(description='Register Professor')

# Create an output widget to display registration status
registration_output = widgets.Output()

# Define a function to handle professor registration
def handle_register_professor(button):
    # Extract information from widgets
    professor_info = {
        'identificacion': professor_identification_input.value,
        'nombre': professor_name_input.value,
        'fecha_nacimiento': professor_birthdate_input.value,
        'nacionalidad': professor_nationality_input.value,
        'telefonos': professor_telephones_input.value,
        'emails': professor_emails_input.value,
        'direccion': professor_address_input.value,
        'especialidad': professor_specialty_input.value,
        'anos_ensenanza': professor_teaching_years_input.value,
        'genero_profesor': professor_gender_input.value,
        'fotosUrl': [registered_faces[registered_name]]
    }

    # Create a Professor instance and save it
    professor_instance = Profesor(**professor_info)
    save_instance(professor_instance, 'professor_data.bin')

    # Display registration status
    with registration_output:
        clear_output()
        print(f"Professor {professor_info['nombre']} registered successfully!")

# Attach the function to the button click event
register_professor_button.on_click(handle_register_professor)

# Create a container for the widgets
professor_registration_container = widgets.VBox([
    professor_identification_input,
    professor_name_input,
    professor_birthdate_input,
    professor_nationality_input,
    professor_telephones_input,
    professor_emails_input,
    professor_address_input,
    professor_specialty_input,
    professor_teaching_years_input,
    professor_gender_input,
    register_professor_button,
    registration_output
])

# Display the container
display(professor_registration_container)



#***************************************************************************************************
#***************************************************************************************************
#***************************************************************************************************

add_horizontal_line()
add_horizontal_line()

# Create a button to trigger the professor information query
query_professor_button = widgets.Button(description='Query Professor Information')

# Create an output widget to display the professor information
query_output = widgets.Output()


def handle_query_professor(button):
    with query_output:
        clear_output()

        # Load professor instances from the binary file
        professors_list = load_instances('professor_data.bin')

        if not professors_list:
            print("No professor information found.")
        else:
            for professor_instance in professors_list:
                print(f"Professor ID: {professor_instance.identificacion}")
                print(f"Name: {professor_instance.nombre}")
                print(f"Specialty: {professor_instance.especialidad}")
                print(f"Teaching Years: {professor_instance.anos_ensenanza}")
                print(f"Gender: {professor_instance.genero_profesor}")  # Ajustar este campo si es necesario
                print("\n")



# Attach the function to the button click event
query_professor_button.on_click(handle_query_professor)

# Create a container for the widgets
query_professor_container = widgets.VBox([query_professor_button, query_output])

# Display the container
display(query_professor_container)




#***************************************************************************************************
#***************************************************************************************************
#***************************************************************************************************



add_horizontal_line()
add_horizontal_line()


#***************************************************************************************************
#***************************************************************************************************
#***************************************************************************************************

# A PARTIR DE ESTA LINEA COMENZAMOS A CREAR LOS WIDGETS

# Create a button to display the list of students
show_students_button = widgets.Button(description='Show Students List')

# Create an output widget to display the list of students
students_output = widgets.Output()


# Define a function to handle the button click and display the list of students
def handle_show_students(button):
    with students_output:
        clear_output()
        # Assuming you have a list of student instances, replace `students_list` with your actual list
        students_list = [student_instance]  # Replace this with your actual list of student instances
        for student in students_list:
            print(f"ID: {student.identificacion}, Name: {student.nombre}, Carrera: {student.carrera}")

# Attach the function to the button click event
show_students_button.on_click(handle_show_students)

# Create a container for the widgets
students_container = widgets.VBox([show_students_button, students_output])

# Display the container
display(students_container)


add_horizontal_line()
add_horizontal_line()

#***************************************************************************************************
#***************************************************************************************************
#***************************************************************************************************

# Save a list of instances to a binary file
def save_instances(instances, file):
    with open(file, 'wb') as f:
        pickle.dump(instances, f)



def save_instance(instance, file):
    # Cargar instancias existentes
    instances_list = load_instances(file)

    # Crear una nueva lista si no hay instancias cargadas previamente
    if not instances_list:
        instances_list = []

    # Agregar la nueva instancia a la lista
    instances_list.append(instance)

    # Guardar la lista actualizada
    save_instances(instances_list, file)



# Add a new instance to the list and save it to a binary file
def add_instance(instance, file):
    # Load existing instances
    instances_list = load_instances(file)

    # Create a new list if no instances were loaded previously
    if not instances_list:
        instances_list = []

    # Add the new instance to the list
    instances_list.append(instance)

    # Save the updated list
    save_instances(instances_list, file)




def load_instances(file):
    try:
        with open(file, 'rb') as f:
            # Cargar la lista de instancias desde el archivo binario
            instances_list = pickle.load(f)
        return instances_list
    except FileNotFoundError:
        # Si el archivo no existe, retornar una lista vacía
        return []



# Save example instances to binary files (modify as needed)

# Crear instancia de Profesor
professor_instance = Profesor("ID123", "Profesor1", "01/01/1980", "Nacionalidad1", {"tel1": "123", "cel": "456"}, ["email1@example.com", "email2@example.com"], "Direccion1", "Especialidad1", 10, "Masculino", ["Propiedad1"])

# Almacenar la instancia en una lista
professors_list = [professor_instance]

# Guardar la lista de instancias en un archivo binario
save_instances(professors_list, 'professor_data.bin')


# student_instance = Estudiante("ID456", "Estudiante1", "01/01/2000", "Nacionalidad2", {"tel1": "789", "cel": "012"}, ["email3@example.com", "email4@example.com"], "Direccion2", "Carrera1", "Fútbol", "Aries", ["Propiedad2"])
# save_instance(student_instance, 'student_data.bin')

# janitor_instance = Miscelaneo("ID789", "Miscelaneo1", "01/01/1970", "Nacionalidad3", {"tel1": "111", "cel": "222"}, ["email5@example.com", "email6@example.com"], "Direccion3", "Turno1", "Soltero", "500", ["Propiedad3"])
# save_instance(janitor_instance, 'janitor_data.bin')

# classroom_instance = Aula("Aula101", janitor_instance, ["Propiedad4"])
# save_instance(classroom_instance, 'classroom_data.bin')

# course_instance = Curso("C101", "Curso1", "Periodo1", "Carrera1", [student_instance], [professor_instance], "Lunes 10:00 AM", [classroom_instance], ["Propiedad5"])
# save_instance(course_instance, 'course_data.bin')

# attendance_instance = Asistencia("Estudiante1", "/path/to/image.jpg", "01/01/2023 10:30 AM", ["Propiedad6"])
# save_instance(attendance_instance, 'attendance_data.bin')
